# LLM Prompt Template Model

Autor: Jonas

O modelo não possui:

- Belief Tracker

## Importação de Bibliotecas

Aqui, bibliotecas para os transformers são importadas e a declaração do modelo BERTimbau e Tokenizer são feitas.
Logo em seguida, a importação dos dados no Drive compartilhado são feitos e os dados carregados.

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import torch
import spacy
from transformers import pipeline, BertTokenizer, BertForTokenClassification, BertForQuestionAnswering
from transformers import BertForMaskedLM
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
#from transformers import BertForPreTraining  # Or BertForPreTraining for loading pretraining heads
#from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

model = BertForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more det

In [ ]:
# Montagem do Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carrega o dataset
data_path = '/content/drive/Shareddrives/Grupo_05/base_aug_20240918_v1.xlsx'
df = pd.read_excel(data_path)
df.head()

,Unnamed: 0,Intencao,Texto
0,0,Como depositar,dia gostar caixa family mart verde eh hora tra...
1,1,Como fazer remessa,enviar dinheiro japo
2,2,Tempo de remessa,levar beneficirio receber dinheiro
3,3,"Pedido de envio via metodo ""ByPhone""",acabar transferncia total iene
4,4,"Pedido de envio via metodo ""ByPhone""",poder remesso iene iene caixa econmic...


## Pré-Processamento

O pré-processamento é necessário para aplicar fine tuning no modelo de LLM. Como os templates com fluxo de regras não necessita de LLM com fine tuning próprio, esta etapa foi pulada.

In [ ]:
# Pré-Processamento para o modelo LLM Template

def preprocessing(df):
  df.drop(['Unnamed: 0'], axis=1, inplace=True)
  return df

In [ ]:
df.head()

,Unnamed: 0,Intencao,Texto
0,0,Como depositar,dia gostar caixa family mart verde eh hora tra...
1,1,Como fazer remessa,enviar dinheiro japo
2,2,Tempo de remessa,levar beneficirio receber dinheiro
3,3,"Pedido de envio via metodo ""ByPhone""",acabar transferncia total iene
4,4,"Pedido de envio via metodo ""ByPhone""",poder remesso iene iene caixa econmic...


## Predição de Intenções

O BERTimbau para predição de sequências é utilizado para predizer intenções conforme um texto e utilizar uma lista de intenções pré-estabelecidas para extrair intenções e selecionar de acordo com o pré-estabelecido. Porém, as tentativas de extrair intenções foram falhas, já que ele está selecionando intenções aleatórias.

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Create a pipeline for question answering
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Get user input
user_input = input("Enter your text: ")

# Process the input
encoded_input = tokenizer(user_input, return_tensors='pt')
output = model(**encoded_input)

# Get the predicted label (intent)
predicted_label = output.logits.argmax().item()

# Assuming you have a mapping between labels and intents
intents = ['qual_taxa_conversao', 'qual_tempo_remessa']
predicted_intent = intents[predicted_label]

print("Intenção predita: ", predicted_intent)

## Identificação de Entidades

Testes são feitos para usar o BERTimbau para classificação de tokens com o objetivo de identificar entidades num texto e extrair-las a parte para futuro uso em templates que possuem informações mais detalhas sobre elas. Porém, o BERTimbau somente retorna labels genéricas, não sendo frutífero para ser utilizado.

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load pre-trained BERT tokenizer and model for NER
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

# Create a pipeline for NER
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Get user input
user_input = "Seu nome é Roberto"

# Process the input and extract entities
result = nlp(user_input)

# Print the extracted entities
for entity in result:
    print(entity)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entity': 'LABEL_1', 'score': 0.56299585, 'index': 1, 'word': 'Seu', 'start': None, 'end': None}
{'entity': 'LABEL_1', 'score': 0.5330809, 'index': 2, 'word': 'nome', 'start': None, 'end': None}
{'entity': 'LABEL_1', 'score': 0.5046829, 'index': 3, 'word': 'é', 'start': None, 'end': None}
{'entity': 'LABEL_1', 'score': 0.6092706, 'index': 4, 'word': 'Roberto', 'start': None, 'end': None}


## Respondendo Perguntas

Outro modelo do BERTimbau é utilizado para testar sua capacidade de responder perguntas para geração de contexto para o modelo, mas suas respostas, na maioria das vezes, não apresenta sentido lógico, somente retornando partes do texto original.

In [ ]:
gen = pipeline("question-answering", model=BertForQuestionAnswering.from_pretrained("neuralmind/bert-base-portuguese-cased"), tokenizer=tokenizer)

question = "Olá, qual é seu nome?"
context = "O seu nome é Roberto"
result = gen(question = question, context = context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Answer: 'O seu nome é Roberto', score: 0.0332, start: 0, end: 19


## Prompt Engineering

A Gemini AI API é utilizada para tentativa de prompt engineering com templates de resposta para a AI. Um fluxo de regras simples é adotado e o Gemini é capaz de incorporar um chatbot com respostas simples baseadas num template.
Ao inves do Gemini, o modelo pré-treinado com fine tuning de outros membros pode ser utilizado em cima para gerar respostas baseadas em LLM para casos mais complexos, incorporando Belief Tracker.

In [ ]:
# Instalação AI generative Google

!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 16.4 MB/s eta 0:00:00


In [ ]:
# Importação de bibliotecas

import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', ' *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# api_key do Gemini (REMOVIDO)

genai.configure(api_key='')

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro')

response = model.generate_content("O que é a Brastel?") # Teste da AI.
to_markdown(response.text)

> A Brastel é uma empresa japonesa de telecomunicações que oferece **serviços de chamadas internacionais de baixo custo**. Ela permite que pessoas no Japão façam ligações internacionais baratas para telefones fixos e celulares em todo o mundo. 
> 
> **Alguns dos serviços oferecidos pela Brastel incluem:**
> 
> * **Cartões telefônicos pré-pagos:** Permitem que os usuários façam chamadas internacionais baratas de qualquer telefone fixo ou celular.
> * **Aplicativo móvel:** Permite que os usuários façam chamadas internacionais através da internet, usando seus smartphones.
> * **Números de acesso local:** Permitem que os usuários façam chamadas internacionais a partir de telefones fixos sem precisar discar um número 00xx.
> 
> A Brastel é uma opção popular para estrangeiros que vivem no Japão e precisam fazer chamadas internacionais frequentes. Seus serviços são conhecidos por serem confiáveis e acessíveis.
> 
> **Aqui estão alguns dos benefícios de usar a Brastel:**
> 
> * **Tarifas baixas para chamadas internacionais.**
> * **Fácil de usar -** Cartões pré-pagos, aplicativo móvel e números de acesso local.
> * **Confiável -** Boa qualidade de chamada.
> * **Disponível em vários idiomas.**
> 
> Espero que esta informação seja útil! 


In [ ]:
# Definição de user input e prompt engineering.

user_input = "Qual a taxa de câmbio?"

templates = {"qual_taxa_conversao": "A taxa de conversão é {taxa}.",
             "qual_tempo_remessa": "O tempo de remessa é {taxa}.",
             "qual_taxa_cambio": "A taxa de câmbio da {moeda1} e {moeda2} é {taxa_cambio}"
}

entities = {
    "taxa": "3.12%",
    "moeda1": "Yen",
    "moeda2": "Real",
    "tempo_remessa": "2 horas",
    "taxa_cambio": "0.6"
}

if "câmbio" in user_input:
  intention = "qual_taxa_cambio"

print(templates[intention].format(**entities))

A taxa de câmbio da Yen e Real é 0.6
